In [37]:
import sqlite3

connection = sqlite3.connect("schema.db")
cursor = connection.cursor()

# Drop existing tables if they exist
drop_ddl = [
    "DROP TABLE IF EXISTS \"association\";",
    "DROP TABLE IF EXISTS \"directed_relationship\";",
    "DROP TABLE IF EXISTS \"relationship\";",
    "DROP TABLE IF EXISTS \"primitive\";",
    "DROP TABLE IF EXISTS \"interface\";",
    "DROP TABLE IF EXISTS \"namespace\";",
    "DROP TABLE IF EXISTS \"internal_enumeration\";",
    "DROP TABLE IF EXISTS \"interface_uuid\";",
    "DROP TABLE IF EXISTS \"dependency\";"
]

# Execute drop table commands
for statement in drop_ddl:
    cursor.execute(statement)

# SQL commands for creating new tables
create_ddl = [
    """
    CREATE TABLE IF NOT EXISTS "internal_enumeration"(
        "id" INTEGER NOT NULL,
        "value" TEXT NOT NULL
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "namespace"(
        "id" INTEGER PRIMARY KEY AUTOINCREMENT,
        "name" TEXT,
        "parent" INTEGER,
        FOREIGN KEY("parent") REFERENCES "namespace"("id")
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "interface"(
        "id" INTEGER PRIMARY KEY AUTOINCREMENT,
        "namespace" INTEGER NOT NULL,
        "name" TEXT,
        FOREIGN KEY("namespace") REFERENCES "namespace"("id")
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "primitive"(
        "id" INTEGER PRIMARY KEY,
        FOREIGN KEY("id") REFERENCES "interface"("id")
    ) WITHOUT ROWID;
    """,
    """
    CREATE TABLE IF NOT EXISTS "relationship"(
        "id" INTEGER PRIMARY KEY AUTOINCREMENT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "directed_relationship"(
        "id" INTEGER PRIMARY KEY,
        "source" INTEGER NOT NULL,
        "target" INTEGER NOT NULL,
        FOREIGN KEY("id") REFERENCES "relationship"("id"),
        FOREIGN KEY("source") REFERENCES "interface"("id"),
        FOREIGN KEY("target") REFERENCES "interface"("id")
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS "association"(
        "id" INTEGER PRIMARY KEY,
        "name" TEXT,
        "domain" INTEGER NOT NULL,
        "range" INTEGER NOT NULL,
        "required" INTEGER DEFAULT 'FALSE',
        "multi" INTEGER DEFAULT 'FALSE',
        "exclusive" INTEGER DEFAULT 'FALSE',
        FOREIGN KEY("id") REFERENCES "relationship"("id"),
        FOREIGN KEY("domain") REFERENCES "interface"("id"),
        FOREIGN KEY("range") REFERENCES "interface"("id")
    );
    """
]

# Execute create table commands
for statement in create_ddl:
    cursor.execute(statement)

# Commit changes and close the connection
connection.commit()
connection.close()

In [38]:
import sqlite3

connection = sqlite3.connect("std.db")
cursor = connection.cursor()

cursor.execute("""INSERT INTO "namespace" ("name") VALUES ('std');""")
std_namespace_id = cursor.lastrowid

primitives = ['Integer32', 'Boolean', 'String', 'Float64']
primitive_ids = []

for primitive in primitives:
    cursor.execute("""INSERT INTO "interface" ("namespace", "name") VALUES (?, ?);""", (std_namespace_id, primitive))
    primitive_interface_id = cursor.lastrowid
    primitive_ids.append(primitive_interface_id) 
    cursor.execute("""INSERT INTO "primitive" ("id") VALUES (?);""", (primitive_interface_id,))

primitives = dict(zip(primitives, primitive_ids))

connection = sqlite3.connect("legal.db")
cursor = connection.cursor()

cursor.execute("""INSERT INTO "namespace" ("name") VALUES ('legal');""")
legal_namespace_id = cursor.lastrowid

cursor.execute("""INSERT INTO "interface" ("namespace", "name") VALUES (?, 'LegalSubject');""", (legal_namespace_id,))
legal_subject_interface_id = cursor.lastrowid

connection.commit()
connection.close()

connection = sqlite3.connect("person.db")

cursor.execute("""INSERT INTO "namespace" ("name") VALUES ('person');""")
person_namespace_id = cursor.lastrowid

cursor.execute("""INSERT INTO "interface" ("namespace", "name") VALUES (?, 'Person');""", (person_namespace_id,))
person_interface_id = cursor.lastrowid

# Insert relationships
cursor.execute("""INSERT INTO "relationship" DEFAULT VALUES;""")
relationship_id = cursor.lastrowid
cursor.execute(
    """INSERT INTO "directed_relationship" ("id", "source", "target") VALUES (?, ?, ?);""",
    (relationship_id, person_interface_id, legal_subject_interface_id)
)

# Create association
cursor.execute("""INSERT INTO "relationship" DEFAULT VALUES;""")
relationship_id = cursor.lastrowid
cursor.execute(
    """INSERT INTO "association" ("id", "name", "domain", "range", "required", "multi", "exclusive") VALUES (?, ?, ?, ?, 'FALSE', 'FALSE', 'FALSE');""",
    (relationship_id, "age", person_interface_id, primitives["Integer32"])
)

connection.commit()
connection.close()

In [39]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("schema.db")
cursor = connection.cursor()

query = """
    SELECT n.name, i.name, i.id
    FROM interface i
    JOIN namespace n ON i.namespace = n.id
"""

resultset_df = pd.read_sql(query, connection)

connection.close()

resultset_df

,name,name,id
0,std,Integer32,1
1,std,Boolean,2
2,std,String,3
3,std,Float64,4
4,legal,LegalSubject,5
5,person,Person,6


In [41]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("schema.db")
cursor = connection.cursor()

# List all properties of an interface.
query = """
    SELECT 
        id.name AS interface,
        a.name AS association, 
        ir.name AS range,
        a.required, 
        a.multi, 
        a.exclusive
    FROM association a
    JOIN interface id ON a.domain = id.id
    JOIN interface ir ON a.range = ir.id
    WHERE id.name = 'Person';
"""

resultset_df = pd.read_sql(query, connection)
 
connection.close()

resultset_df

,interface,association,range,required,multi,exclusive
0,Person,age,Integer32,FALSE,FALSE,FALSE
